# CNN
- 합성곱 계층: 사진의 뉴런이 늘어남 (Convolution)
- 풀링 계층: 사이즈를 줄임 (Pooling)
- Feature Extraction & Classification

In [ ]:
import cv2
img = cv2.imread('./images/digits.png', 0)
print(img.shape)

In [ ]:
import numpy as np
import pandas as pd

cells = [np.hsplit(row, 100) for row in np.vsplit(img, 50)]
x = np.array(cells)/255.
img_rows, img_cols = 20, 20
X = x.reshape(-1, img_rows, img_cols, 1)
y = pd.get_dummies(np.repeat(np.arange(10), 500)).to_numpy()

from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)

print(train_X.shape, train_y.shape)

In [ ]:
from tensorflow.keras.models import Sequential

model = Sequential()

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input, Dropout, MaxPooling2D

model.add(Conv2D(32, kernel_size=(3,3), input_shape=(img_rows, img_cols, 1), activation='relu')) #28, 28, 1
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.1))

model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))


In [ ]:
from tensorflow.keras.optimizers import Adam

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['acc'])

In [ ]:
hist = model.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=200, batch_size=128, verbose=1)

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax = loss_ax.twinx()
acc_ax.plot(hist.history['acc'], 'b', label ='train acc')
acc_ax.plot(hist.history['val_acc'], 'b', label ='val acc')
acc_ax.set_ylabel('accuracy')
# acc_ax.set_legend(loc='lower left')

plt.show()

In [ ]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)

if cap.isOpened():
    while True:
        ret, img = cap.read()
        if ret:
            g_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            _, bin_img = cv2.threshold(g_img, 110, 255, cv2.THRESH_BINARY_INV)
            contours, hierarchy = cv2.findContours(bin_img, 
                        cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            # CHAIN_APPROX_SIMPLE 테두리를 잡아줌 
            try:
                for i in range(len(contours)):
                    contour = contours[i]
                    (x,y), radius = cv2.minEnclosingCircle(contour)
                    if radius > 3:
                        xs, xe = int(x-radius), int(x+radius)
                        ys, ye = int(y-radius), int(y+radius)
                        cv2.rectangle(bin_img, (xs, ys), (xe, ye), (200, 0, 0), 1)
                        roi = bin_img[ys:ye, xs:xe]
                        dst = cv2.resize(roi, dsize = (50, 50))
                        dst = cv2.resize(dst, dsize = (24, 24)) #16, 16
                        A = np.zeros((28,28))  # 20, 20
                        A[2:-2, 2:-2] = dst[:,:]
                        A = A.reshape(-1,28, 28, 1)  #-1, 20, 20, 1
                        num =np.argmax(model.predict(A))
                        cv2.putText(bin_img, str(num), (xs,ys), cv2.FONT_HERSHEY_PLAIN, 2, (200, 0, 0))
            except Exception as e:
                print(e)
            cv2.imshow("Image", bin_img)
            if cv2.waitKey(1) & 0xFF ==27:
                break
        else:
            print("Np Frame")
            break
else:
    print("Camera not opened")

cap.release()
cv2.destroyAllWindows()

### Mnist

In [ ]:
from tensorflow.keras.datasets import mnist
(train_img, train_labels), (test_img, test_labels) = mnist.load_data()

In [ ]:
#X의 모양을 입력 모양에 맞도록 수정

train_img = train_img.reshape(-1, 28, 28, 1)
test_img = test_img.reshape(-1, 28, 28, 1)

#표준화
train_img = train_img.astype('float32')/255
test_img = test_img.astype('float32') / 255

#y를 원-핫 인코딩 

from tensorflow.keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
model.fit(train_img, train_labels, validation_data=(test_img, test_labels), epochs=20, batch_size=128, verbose=1)

## Model 저장
### ModelCheckpoint

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
x_train, x_test = x_train/255.0, x_test/255.0

In [ ]:
MODEL_SAVE_FOLDER = './model/'
import os
if not os.path.exists(MODEL_SAVE_FOLDER):
    os.mkdir(MODEL_SAVE_FOLDER)
path = MODEL_SAVE_FOLDER + '{epoch:02d}-{val_acc:.4f}.hdf5'
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True)



In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(360, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['acc'])

model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=1000, epochs=50, verbose=1, callbacks=[checkpoint])

In [ ]:
model.evaluate(x_test, y_test)

### EarlyStopping Callback


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train/255.0, x_test/255.0

In [2]:
MODEL_SAVE_FOLDER = './model/'
import os
if not os.path.exists(MODEL_SAVE_FOLDER):
    os.mkdir(MODEL_SAVE_FOLDER)
path = MODEL_SAVE_FOLDER + '{epoch:02d}-{val_acc:.4f}.hdf5'
checkpoint = ModelCheckpoint(path, monitor='val_acc', verbose=1, save_best_only=True)
#val_accuracy가 3번 이상 증가하지 않으면 학습을 중단
early_stopping = EarlyStopping(monitor='val_acc', patience=3)




In [3]:
from tensorflow.keras.callbacks import TensorBoard
# histogram_freg가 1이면 epoch이 1마다 저장됨
tensor_board = TensorBoard(log_dir = './tensor_log', embeddings_freq=1, histogram_freq=1)

In [4]:
model = Sequential()
model.add(Flatten(input_shape=(28,28)))
model.add(Dense(360, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['acc'])


#checkpoint(모델), early_stopping(가중치)를 동시에 저장함
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=1000, epochs=1000, 
          verbose=1, callbacks=[checkpoint, early_stopping, tensor_board])

Epoch 1/1000
60/60 [==============================] - 2s 25ms/step - loss: 2.2186 - acc: 0.2008 - val_loss: 1.8264 - val_acc: 0.6207

Epoch 00001: val_acc improved from -inf to 0.62070, saving model to ./model\01-0.6207.hdf5
Epoch 2/1000
60/60 [==============================] - 1s 12ms/step - loss: 1.7499 - acc: 0.5788 - val_loss: 1.3960 - val_acc: 0.7293

Epoch 00002: val_acc improved from 0.62070 to 0.72930, saving model to ./model\02-0.7293.hdf5
Epoch 3/1000
60/60 [==============================] - 1s 11ms/step - loss: 1.3659 - acc: 0.6817 - val_loss: 1.0751 - val_acc: 0.7933

Epoch 00003: val_acc improved from 0.72930 to 0.79330, saving model to ./model\03-0.7933.hdf5
Epoch 4/1000
60/60 [==============================] - 1s 11ms/step - loss: 1.0922 - acc: 0.7364 - val_loss: 0.8654 - val_acc: 0.8276

Epoch 00004: val_acc improved from 0.79330 to 0.82760, saving model to ./model\04-0.8276.hdf5
Epoch 5/1000
60/60 [==============================] - 1s 12ms/step - loss: 0.9091 - acc: 0.

Epoch 37/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.3217 - acc: 0.9079 - val_loss: 0.2766 - val_acc: 0.9220

Epoch 00037: val_acc improved from 0.92170 to 0.92200, saving model to ./model\37-0.9220.hdf5
Epoch 38/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.3240 - acc: 0.9070 - val_loss: 0.2739 - val_acc: 0.9234

Epoch 00038: val_acc improved from 0.92200 to 0.92340, saving model to ./model\38-0.9234.hdf5
Epoch 39/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.3200 - acc: 0.9074 - val_loss: 0.2712 - val_acc: 0.9241

Epoch 00039: val_acc improved from 0.92340 to 0.92410, saving model to ./model\39-0.9241.hdf5
Epoch 40/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.3158 - acc: 0.9085 - val_loss: 0.2686 - val_acc: 0.9247

Epoch 00040: val_acc improved from 0.92410 to 0.92470, saving model to ./model\40-0.9247.hdf5
Epoch 41/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.3133 -


Epoch 00073: val_acc improved from 0.93990 to 0.94040, saving model to ./model\73-0.9404.hdf5
Epoch 74/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.2422 - acc: 0.9307 - val_loss: 0.2099 - val_acc: 0.9405

Epoch 00074: val_acc improved from 0.94040 to 0.94050, saving model to ./model\74-0.9405.hdf5
Epoch 75/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.2419 - acc: 0.9289 - val_loss: 0.2086 - val_acc: 0.9411

Epoch 00075: val_acc improved from 0.94050 to 0.94110, saving model to ./model\75-0.9411.hdf5
Epoch 76/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.2467 - acc: 0.9287 - val_loss: 0.2074 - val_acc: 0.9413

Epoch 00076: val_acc improved from 0.94110 to 0.94130, saving model to ./model\76-0.9413.hdf5
Epoch 77/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.2396 - acc: 0.9298 - val_loss: 0.2063 - val_acc: 0.9420

Epoch 00077: val_acc improved from 0.94130 to 0.94200, saving model to ./model\77-0

Epoch 112/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.2015 - acc: 0.9415 - val_loss: 0.1726 - val_acc: 0.9501

Epoch 00112: val_acc improved from 0.94980 to 0.95010, saving model to ./model\112-0.9501.hdf5
Epoch 113/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.1973 - acc: 0.9441 - val_loss: 0.1720 - val_acc: 0.9505

Epoch 00113: val_acc improved from 0.95010 to 0.95050, saving model to ./model\113-0.9505.hdf5
Epoch 114/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.1952 - acc: 0.9439 - val_loss: 0.1711 - val_acc: 0.9506

Epoch 00114: val_acc improved from 0.95050 to 0.95060, saving model to ./model\114-0.9506.hdf5
Epoch 115/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.2019 - acc: 0.9412 - val_loss: 0.1705 - val_acc: 0.9506

Epoch 00115: val_acc did not improve from 0.95060
Epoch 116/1000
60/60 [==============================] - 1s 11ms/step - loss: 0.1917 - acc: 0.9458 - val_loss: 0.1697 - va

In [ ]:
model.evaluate(x_test, y_test)

## 모델구조 저장하고 불러오기 (json)

In [ ]:
#커널 재시작
from tensorflow.keras.models import load_model
model = load_model('./model/84-0.9412.hdf5')

In [ ]:
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['acc'])

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train, x_test = x_train/255.0, x_test/255.0

model.evaluate(x_test, y_test)

In [ ]:
# 모델의 구조만 저장

model_json = model.to_json()
with open ("digits_model.json", 'w') as json_file:
    json_file.write(model_json)

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.models import model_from_json
with open ("digits_model.json", 'r') as f:
    load_model_json=f.read()
    model = model_from_json(load_model_json)
    
model.load_weights('./model/84-0.9412.hdf5')

In [ ]:
#모델의 구조와 가중치를 따로 불러왔기 때문에 컴파일을 다시 해주어야함
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['acc'])

model.evaluate(x_test, y_test)